In [23]:
!pip install Pyfhel
!pip install pynacl
!pip install cryptography
!pip install tqdm

In [24]:
no_clients = 3
epochs = 3

In [25]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

# Check if GPU is available
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    print("GPUs available:", len(gpus))
    for gpu in gpus:
        print(gpu)
else:
    print("No GPU available.")

TensorFlow version: 2.14.1
GPUs available: 1
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [26]:
import tensorflow as tf
from tqdm import tqdm
import copy
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import pickle
import sys
import matplotlib.pyplot as plt
import numpy as np

In [27]:
import numpy as np
import os
import tensorflow as tf

save_dir = "dataset/mnist_data/"
os.makedirs(save_dir, exist_ok=True)


def download_and_save_mnist(save_dir):
    (x_train_all, y_train_all), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    np.save(os.path.join(save_dir, "x_train.npy"), x_train_all)
    np.save(os.path.join(save_dir, "y_train.npy"), y_train_all)
    np.save(os.path.join(save_dir, "x_test.npy"), x_test)
    np.save(os.path.join(save_dir, "y_test.npy"), y_test)
    print("Dataset downloaded and saved locally.")


def load_mnist_from_local(save_dir):
    x_train_all = np.load(os.path.join(save_dir, "x_train.npy"))
    y_train_all = np.load(os.path.join(save_dir, "y_train.npy"))
    x_test = np.load(os.path.join(save_dir, "x_test.npy"))
    y_test = np.load(os.path.join(save_dir, "y_test.npy"))
    print("Dataset loaded from local files.")
    return (x_train_all, y_train_all), (x_test, y_test)


if not os.path.exists(os.path.join(save_dir, "x_train.npy")):
    download_and_save_mnist(save_dir)

(x_train_all, y_train_all), (x_test, y_test) = load_mnist_from_local(save_dir)

print(f"x_train_all shape: {x_train_all.shape}")
print(f"y_train_all shape: {y_train_all.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")


x_train_all = x_train_all.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

Dataset loaded from local files.
x_train_all shape: (60000, 28, 28)
y_train_all shape: (60000,)
x_test shape: (10000, 28, 28)
y_test shape: (10000,)


In [28]:
def create_model():
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Conv2D(
                32, (3, 3), activation="relu", input_shape=(28, 28, 1)
            ),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(10),
        ]
    )
    return model

In [29]:
class FMLEE:
    def __init__(self, no_clients, epochs):
        self.no_clients = no_clients
        self.epochs = epochs
        self.clients = []
        self.init_clients()
        
    def init_model(self):
        model = tf.keras.models.Sequential(
            [
                tf.keras.layers.Conv2D(
                    32, (3, 3), activation="relu", input_shape=(28, 28, 1)
                ),
                tf.keras.layers.MaxPooling2D((2, 2)),
                tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
                tf.keras.layers.MaxPooling2D((2, 2)),
                tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(64, activation="relu"),
                tf.keras.layers.Dense(10),
            ]
        )
        return model
    
    def init_clients(self):
        for i in range(self.no_clients):
            self.clients.append(self.init_model())
            print(f"Client {i} initialized.")

In [30]:
fml = FMLEE(no_clients, epochs)

Client 0 initialized.


Client 1 initialized.
Client 2 initialized.


In [31]:
fml.clients

In [1]:
from src.data.load_data import load_mnist

2024-06-16 12:45:23.116450: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 12:45:23.116486: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 12:45:23.116517: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-16 12:45:23.122996: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-16 12:45:23.767920: W tensorflow/compiler/

In [2]:
(x_train_all, y_train_all), (x_test, y_test)  = load_mnist()

Saving y_test.npy: 100%|██████████| 10000/10000 [00:00<00:00, 54260077.62it/s]

Dataset downloaded and saved locally at dataset/mnist_data/
Dataset loaded from local files at dataset/mnist_data/
